In [11]:
'''
Website: http://fsecurify.com
'''

'\nWebsite: http://fsecurify.com\n'

In [2]:
import os
import urllib.parse
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [3]:
def loadFile(name):
    directory = str(os.getcwd())
    filepath = os.path.join(directory, name)
    with open(filepath,'r') as f:
        data = f.readlines()
    data = list(set(data))
    result = []
    for d in data:
        d = str(urllib.parse.unquote(d))   #converting url encoded data to simple string
        result.append(d)

    return result

In [4]:
badQueries = loadFile('badqueries.txt')
validQueries = loadFile('goodqueries.txt')

In [5]:
badQueries = list(set(badQueries))
validQueries = list(set(validQueries))
queries = badQueries + validQueries

yBad = [1 for i in range(0, len(badQueries))]  #labels, 1 for malicious and 0 for clean
yGood = [0 for i in range(0, len(validQueries))]
y = yBad + yGood

In [14]:
badQueries[0]

'/9ko6m4c8.kspx?<script>document.cookie="testylik=3403;"</script>\n'

In [13]:
validQueries[:5]

['/javascript/insert.cs\n',
 '/225500/\n',
 '/trasporti_trasloco/\n',
 '/0693543334/\n',
 '/randomradio/\n']

> what is this???

In [6]:
# converting data to vectors

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3))
vectorizer.fit(queries)
X =  vectorizer.transform(queries)

In [15]:
print (X.shape)

(1310506, 97832)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #splitting data

badCount = len(badQueries)
validCount = len(validQueries)
print ("size of good and bad :", validCount, badCount)

size of good and bad : 1265974 44532


In [8]:
# training our model

lgs = LogisticRegression(class_weight={1: 2 * validCount / badCount, 0: 1.0}) # class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight={1: 56.856822060540736, 0: 1.0},
          dual=False, fit_intercept=True, intercept_scaling=1,
          max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2',
          random_state=None, solver='liblinear', tol=0.0001, verbose=0,
          warm_start=False)

In [9]:
##############
# Evaluation #
##############

predicted = lgs.predict(X_test)

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

print ("Bad samples : %d" % badCount)
print ("Good samples: %d" % validCount)
print ("Baseline Constant negative: %.6f" % (validCount / (validCount + badCount)))
print ("------------")
print ("Accuracy    : %f" % lgs.score(X_test, y_test))  #checking the accuracy
print ("Precision   : %f" % metrics.precision_score(y_test, predicted))
print ("Recall      : %f" % metrics.recall_score(y_test, predicted))
print ("F1-Score    : %f" % metrics.f1_score(y_test, predicted))
print ("AUC         : %f" % auc)

Bad samples : 44532
Good samples: 1265974
Baseline Constant negative: 0.966019
------------
Accuracy    : 0.999397
Precision   : 0.983741
Recall      : 0.998520
F1-Score    : 0.991075
AUC         : 0.999974
